# Survival Analysis On Synthesis Data for Diabetic Patients

### Project Details

This is the BYOP (bring your own project) Capstone Project for our IPBA Program from IIM Indore.

### Scope of the Project

This project aims to analyze daibetes patient's TIME ON THERAPY using advance analysis techniques.

### Objective

Main objective for this project is to provide a solution that busineses can implement to understand the patient's behaviour and different scenarios.

### Data

We have generated the simulated Anonymised Patient Level Data using the python Faker module.

| Feild                     | Description                                                                   |
|:-------------------------:|:-----------------------------------------------------------------------------:|
|: claim_id                :|: Insurance Claim Number (Unique)                                             :|
|: patient_id              :|: Patient Unique ID                                                           :|
|: service_date            :|: Date when the claim was initiated                                           :|
|: claim_type              :|: Paid, Rejected or Reversed Claim                                            :|
|: days_supply             :|: No. of Days Drugs was Supplied For                                          :|
|: quantity                :|: Quantity of Medicine(Capsules)                                              :|
|: patient_birth_year      :|: DOB of Patient                                                              :|
|: gender                  :|: Gender of Patient                                                           :|
|: diseases_type           :|: T1: Type 1 / T2: Type 2 Diabetes                                            :|
|: ndc                     :|: Product ID / Unique Durg Code                                               :|
|: drug_name               :|: Brand Name of the Drug                                                      :|
|: class                   :|: Category of Drug                                                            :|
|: sub_class               :|: Sub Category of Drug                                                        :|
|: mode_of_dispensation    :|: How drug is Dispensed - Oral or Injectible                                  :|
|: treatment_hierarchy     :|: The order in which drug is prescribed as diabetes intensifies               :|
|: past_medical_history    :|: Yes or No                                                                   :|
|: family_history          :|: Family History of Diabetes - Yes or No                                      :|
|: is_there_complication   :|: Yes or No                                                                   :|
|: marital_status          :|: Married or Unmarried                                                        :|
|: educational_status      :|: Educated or Uneducated                                                      :|
|: employee_status         :|: Employed or Unemployed                                             :|